In [90]:
from pymongo import MongoClient
import pprint
MONGO_ATLAS_URI = open('.env','r').read()
client = MongoClient(MONGO_ATLAS_URI)
db = client['push']
goals = db['goals']
users = db['users']
goals_array = list(goals.find())
users_array = list(users.find())

print (goals_array)
print (users_array)

[{'_id': ObjectId('5e9fbe235b2f8000240044a0'), 'cadence': 'Daily', 'cadenceCount': 1, 'creationDate': '2020-04-22T03:46:43.699Z', 'creatorID': ObjectId('5e9fb10585928b0024be726c'), 'timeStamps': ['2020-04-21T23:58:05-04:00', '2020-04-22T15:11:05-04:00', '2020-04-23T14:54:17-04:00', '2020-04-25T21:16:01-04:00', '2020-04-25T21:16:02-04:00', '2020-04-25T21:16:02-04:00', '2020-04-25T21:16:03-04:00', '2020-04-29T22:01:17-04:00', '2020-04-30T09:50:21-04:00', '2020-05-03T21:27:47-04:00', '2020-05-03T21:27:50-04:00', '2020-05-03T21:27:53-04:00', '2020-05-03T21:28:58-04:00', '2020-05-03T21:29:00-04:00', '2020-05-03T21:29:02-04:00', '2020-05-03T21:29:04-04:00', '2020-05-17T21:18:35-04:00', '2020-05-18T17:46:49-04:00', '2020-05-20T20:09:15-04:00', '2020-05-20T21:23:48-04:00', '2020-05-22T16:42:14-04:00', '2020-05-23T17:01:46-04:00', '2020-05-23T17:01:55-04:00', '2020-05-25T15:47:10-04:00', '2020-05-26T21:31:27-04:00', '2020-05-28T12:39:04-04:00', '2020-05-31T22:50:59-04:00', '2020-06-01T14:12:59-

In [91]:
def time_stamp_dict_create(time_stamp,record):
    tups = list(record.items()) + [('time_stamp',time_stamp)]
    return dict(tups)


def time_stamp_array_create_from_goals_array(goals_array):
    time_stamps_array_grouped = [[time_stamp_dict_create(time_stamp,D) for time_stamp in D['timeStamps']] for D in goals_array]
    time_stamp_array = [item for sublist in time_stamps_array_grouped for item in sublist]
    return time_stamp_array

time_stamp_array = time_stamp_array_create_from_goals_array(goals_array)
print (time_stamp_array)


[{'_id': ObjectId('5e9fbe235b2f8000240044a0'), 'cadence': 'Daily', 'cadenceCount': 1, 'creationDate': '2020-04-22T03:46:43.699Z', 'creatorID': ObjectId('5e9fb10585928b0024be726c'), 'timeStamps': ['2020-04-21T23:58:05-04:00', '2020-04-22T15:11:05-04:00', '2020-04-23T14:54:17-04:00', '2020-04-25T21:16:01-04:00', '2020-04-25T21:16:02-04:00', '2020-04-25T21:16:02-04:00', '2020-04-25T21:16:03-04:00', '2020-04-29T22:01:17-04:00', '2020-04-30T09:50:21-04:00', '2020-05-03T21:27:47-04:00', '2020-05-03T21:27:50-04:00', '2020-05-03T21:27:53-04:00', '2020-05-03T21:28:58-04:00', '2020-05-03T21:29:00-04:00', '2020-05-03T21:29:02-04:00', '2020-05-03T21:29:04-04:00', '2020-05-17T21:18:35-04:00', '2020-05-18T17:46:49-04:00', '2020-05-20T20:09:15-04:00', '2020-05-20T21:23:48-04:00', '2020-05-22T16:42:14-04:00', '2020-05-23T17:01:46-04:00', '2020-05-23T17:01:55-04:00', '2020-05-25T15:47:10-04:00', '2020-05-26T21:31:27-04:00', '2020-05-28T12:39:04-04:00', '2020-05-31T22:50:59-04:00', '2020-06-01T14:12:59-

In [92]:
from datetime import datetime,timedelta, date
from dateutil.parser import parse
import dateutil.parser
from dateutil.relativedelta import relativedelta

def calculate_month_range(dt):
    month = dt.strftime('%m')
    year = dt.strftime('%Y')
    beginning_of_month_text = "{}-{}-{}".format(str(year),str(month),'01')
    beginning_of_month = parse(beginning_of_month_text)
    next_month_beginning = beginning_of_month + relativedelta(months=1)
    end_of_month = next_month_beginning - timedelta(days=1)
    return [beginning_of_month,end_of_month]

def calculate_week_range(dt):
    start = dt - timedelta(days=dt.weekday()) - timedelta(days=1)
    end = start + timedelta(days=6)
    return [start , end]

def cadence_to_date_range(cadence):
    today = str(date.today())
    dt = parse(str(today))
    if cadence == 'Daily':
        return [dt,dt] 
    elif cadence == 'Weekly':
        return calculate_week_range(dt)
    elif cadence == 'Monthly':
        return calculate_month_range(dt)

    

In [93]:
def determine_time_stamp_in_range(start,end,time_stamp):
    time_stamp = parse(time_stamp).strftime('%Y-%m-%d')
    start = start.strftime('%Y-%m-%d')
    end = end.strftime('%Y-%m-%d')
    return start <= time_stamp <= end

def determine_goal_overdue(goal_record):
    start,end = cadence_to_date_range(goal_record['cadence'])
    time_stamps = goal_record['timeStamps']
    filtered_time_stamps = [time_stamp for time_stamp in time_stamps if determine_time_stamp_in_range(start,end,time_stamp)]
    is_overdue = len(filtered_time_stamps) == 0
    return is_overdue

def overdue_goals_calculate(goals_array):
    overdue_goals = [i for i in goals_array if determine_goal_overdue(i)]
    return overdue_goals 

def completion_percentage_calculate(goals_array):
    overdue_goals = overdue_goals_calculate(goals_array)
    overdue_goals_count = len(overdue_goals)
    goals_count = len(goals_array)
    complete_goals_count = goals_count - overdue_goals_count
    completion_percentage = complete_goals_count / goals_count
    return completion_percentage
overdue_goals = [i for i in goals_array if determine_goal_overdue(i)]

In [94]:
import requests
payload = """{
    "to":"ExponentPushToken[6smFQ5K5I3kM6QNiJTZGAb]",
    "title":"title",
    "body":"abc",
    "sound": "default",
    "_displayInForeground": "true"
}"""

headers = {
  'Content-Type': 'application/json'
}
requests.post('https://exp.host/--/api/v2/push/send',data=payload,headers=headers)

<Response [200]>

In [95]:
users_with_expo_push_tokens = [user for user in users.find() if 'expoPushToken' in user.keys()]
users_with_expo_push_tokens_ids = [user['_id'] for user in users_with_expo_push_tokens]
goals_of_users_with_token = [goal for goal in goals.find() if goal['creatorID'] in users_with_expo_push_tokens_ids]

In [96]:

def goal_latest_time_stamp(goal_dict):
    time_stamps = goal_dict['timeStamps']
    last_time_stamp = time_stamps[-1:]
    return last_time_stamp[0]

def days_calculate_difference(datetime_start,datetime_end):
    total_minutes = (datetime_end - datetime_start).total_seconds() / 60.0
    total_hours = total_minutes/60
    total_days = total_hours/24
    return total_days

def now_calculate_date_time():
    return datetime.now()

def average_days_since_time_stamp(time_stamp):
    time_stamp_parsed = parse(str(time_stamp), ignoretz=True)
    now_date_time = parse(str(now_calculate_date_time()), ignoretz=True)
    days_old = days_calculate_difference(time_stamp_parsed,now_date_time)
    return days_old

def goal_latest_time_stamp_from_array(goals_array):
    time_stamps_latest = [goal_latest_time_stamp(goal_dict) for goal_dict in goals_array]
    return time_stamps_latest 

def average_calculate_from_list(nums):
    total = sum(nums)
    average = total / len(nums)
    return average


def average_age_calculate_from_time_stamp_array(time_stamps_latest):
    days_old_list = [average_days_since_time_stamp(i) for i in time_stamps_latest]
    average_age = average_calculate_from_list(days_old_list)
    return average_age

def goals_array_filter_for_pushes(goals_array):
    goals_array_filtered = [goal_dict for goal_dict in goals_array if len(goal_dict['timeStamps']) > 0]
    return goals_array_filtered

def goal_average_age_calculate_from_array(goals_array):
    goals_array_filtered = goals_array_filter_for_pushes(goals_array)
    time_stamps_latest = goal_latest_time_stamp_from_array(goals_array_filtered) 
    average_age = average_age_calculate_from_time_stamp_array(time_stamps_latest)
    return average_age

print (goal_average_age_calculate_from_array(goals_array))

# print (sum([1,2]))
# now_date_time = parse(str(date.today()))
# print (average_days_since_time_stamp('2020-06-29T15:35:21-04:00'))

15.736475572898705


In [110]:
import itertools
import operator
import time


def list_group_by(l,i):
    return [list(grp) for key, grp in itertools.groupby(sorted(l, key=operator.itemgetter(i)),key=operator.itemgetter(i))]

def time_stamp_to_unicode(time_stamp):
    time_stamp_dt = parse(str(time_stamp), ignoretz=True)
    return time.mktime(time_stamp_dt.timetuple())

def time_stamp_array_sort(time_stamps):
    return sorted(time_stamps, key=time_stamp_to_unicode,reverse=True)
    
def latest_push_from_user(user_goals_array):
    time_stamps_latest = goal_latest_time_stamp_from_array(user_goals_array)
    time_stamps_latest_sorted = time_stamp_array_sort(time_stamps_latest)
    time_stamp_latest = time_stamps_latest_sorted[0]
    return time_stamp_latest
    
def goals_sublists_user_generate(goals_array):
    goals_array_filtered = goals_array_filter_for_pushes(goals_array)
    user_goals_sublists = list_group_by(goals_array_filtered,'creatorID')
    return user_goals_sublists

def active_users_filter(goals_array):
    user_goals_sublists = goals_sublists_user_generate(goals_array)
    users_latest_time_stamps = [latest_push_from_user(subli) for subli in user_goals_sublists]
    days_old_list = [average_days_since_time_stamp(i) for i in users_latest_time_stamps]
    return days_old_list

def active_users_count_filter_last_30_days(goals_array):
    days_old_list = active_users_filter(goals_array)
    active_last_30_days = [i for i in days_old_list if i < 30]
    return len(active_last_30_days)
def active_users_count_filter_last_n_days(goals_array,n):
    days_old_list = active_users_filter(goals_array)
    active_last_30_days = [i for i in days_old_list if i < n]
    return len(active_last_30_days)
def users_with_goals_calculate(goals_array):
    user_goals_sublists = goals_sublists_user_generate(goals_array)
    users_with_goals_count = len(user_goals_sublists)
    return users_with_goals_count



In [98]:
def users_with_expo_push_token_calculate(users_array):
    users_with_expo_push_tokens = [user for user in users_array if 'expoPushToken' in user.keys()]
    return users_with_expo_push_tokens

In [99]:
def dictionary_combine(D1,D2):
    tups = list(D1.items()) + list(D2.items())
    return dict(tups)
def user_dict_append_missing_keys(user_dict,keys):
    tups = [(key,'-') for key in keys if key not in user_dict.keys()]
    return tups 
            
def users_dict_transform_base(user_dict,keys=['email','createdDate','_id','expoPushToken']):
    key_filtered_tups = [(k,str(v)) for k,v in user_dict.items() if k in keys]
    missing_tups = user_dict_append_missing_keys(user_dict,keys)
    total_tups = key_filtered_tups + missing_tups
    return dict(total_tups)
    
def user_dict_filter_goals_array(user_dict,goals_array):
    user_id = user_dict['_id']
    goals_filtered_array=[goal_dict for goal_dict in goals_array if user_id == goal_dict['creatorID']]
    return goals_filtered_array

def latest_push_from_user_try(goals_array_filtered_pushes):
    try:
        latest_time_stamp = latest_push_from_user(goals_array_filtered_pushes)
        days_since_active = average_days_since_time_stamp(latest_time_stamp)
        return latest_time_stamp,days_since_active
    except:
        return '-','-'


def custom_dict_from_goals(goals_filtered_array,time_stamps_array):
    goal_count = len(goals_filtered_array)
    push_count  = len(goals_filtered_array)
    goals_array_filtered_pushes = goals_array_filter_for_pushes(goals_filtered_array)
    overdue_goals = len(overdue_goals_calculate(goals_array_filtered_pushes))
    goals_complete = goal_count-overdue_goals
    percentage_complete = (goals_complete/goal_count) if goal_count > 0 else 0
    latest_time_stamp,days_since_active = latest_push_from_user_try(goals_array_filtered_pushes)
    return {
        'goals':goal_count,
        'goals_overdue':overdue_goals,
        'goals_complete':goals_complete,
        'percentage_complete':percentage_complete,
        'pushes':push_count,
        'last_active':latest_time_stamp,
        'days_since_active':days_since_active
    }

def custom_dict_from_user(user_dict):
    new_user_dict = users_dict_transform_base(user_dict)
    has_expo_push_token = 'expoPushToken' in user_dict.keys()
    return dictionary_combine(new_user_dict,{'has_push_token':has_expo_push_token})
    
def users_dict_transform(user_dict,goals_array):
    goals_filtered_array = user_dict_filter_goals_array(user_dict,goals_array)
    time_stamps_array = time_stamp_array_create_from_goals_array(goals_filtered_array)
    goal_push_dict = custom_dict_from_goals(goals_filtered_array,time_stamps_array)
    user_dict = custom_dict_from_user(user_dict)
    return dictionary_combine(user_dict,goal_push_dict)

def users_array_transform(users_array,goals_array):
    users_array = [users_dict_transform(user_dict,goals_array) for user_dict in users_array] 
    return users_array 

users_array_transformed = users_array_transform(users_array,goals_array)
print (users_array_transformed[0])

{'_id': '5e9fb10585928b0024be726c', 'email': 'cruzc09@gmail.com', 'createdDate': '2020-04-26T02:46:55.895Z', 'expoPushToken': 'ExponentPushToken[uxbMbWN05bmcWe9V4JYfqX]', 'has_push_token': True, 'goals': 35, 'goals_overdue': 23, 'goals_complete': 12, 'percentage_complete': 0.34285714285714286, 'pushes': 35, 'last_active': '2020-07-20T20:44:10-04:00', 'days_since_active': 1.5574431245370368}


In [111]:
number_of_goals = len(goals_array)
number_of_users = len(users_array)
number_of_pushes = len(time_stamp_array)
number_of_goals_overdue =len(overdue_goals)

percentage_complete = completion_percentage_calculate(goals_array)
active_users_last_30_days = active_users_count_filter_last_n_days(goals_array,7)
users_with_goals = users_with_goals_calculate(goals_array)
users_with_expo_push_token = len(users_with_expo_push_token_calculate(users_array))

number_of_users_text = "Total Users: {}".format(number_of_users)
active_users_last_30_days_text = "Users (Active): {}".format(active_users_last_30_days)
users_with_goals_text = "Users (With Goals): {}".format(users_with_goals)
users_with_expo_push_token_text = "Users (With Push Token): {}".format(users_with_expo_push_token)
number_of_goals_text = "Goals: {}".format(number_of_goals)
number_of_pushes_text = "Pushes: {}".format(number_of_pushes)
percentage_complete_text = "Completion Percentage: {}".format(percentage_complete)
number_of_goals_overdue_text = "Goals Overdue: {}".format(number_of_goals_overdue)


print (number_of_users_text)
print (active_users_last_30_days_text)
print (users_with_goals_text)
print (users_with_expo_push_token_text)
print (number_of_goals_text)
print (number_of_pushes_text)
print (percentage_complete_text)
print (number_of_goals_overdue_text)




Total Users: 39
Users (Active): 10
Users (With Goals): 25
Users (With Push Token): 11
Goals: 125
Pushes: 1445
Completion Percentage: 0.128
Goals Overdue: 109


In [101]:
print (users_array[0])

{'_id': ObjectId('5e9fb10585928b0024be726c'), 'userName': 'Chris', 'email': 'cruzc09@gmail.com', 'password': '$argon2i$v=19$m=4096,t=3,p=1$FSM4sxSdJuy0OeO4znvCrjhKWJVHx01Z/3I3A9PwTP4$DlxVeiBxCLpIV2JVoWqhjgAg2BJ7/Wc6tXORcfQh7CI', 'salt': b'\x15#8\xb3\x14\x9d&\xec\xb49\xe3\xb8\xce{\xc2\xae8JX\x95G\xc7MY\xffr7\x03\xd3\xf0L\xfe', 'createdDate': '2020-04-26T02:46:55.895Z', 'expoPushToken': 'ExponentPushToken[uxbMbWN05bmcWe9V4JYfqX]'}


In [102]:
print (goals_array[2])

{'_id': ObjectId('5eaa3b92878cfb00242a02f9'), 'cadence': 'Weekly', 'cadenceCount': 3, 'creationDate': '2020-04-30T02:44:34.411Z', 'creatorID': ObjectId('5e9fb10585928b0024be726c'), 'timeStamps': ['2020-04-30T00:03:08-04:00', '2020-05-03T16:00:45-04:00', '2020-05-03T16:17:17-04:00', '2020-05-10T13:09:10-04:00', '2020-05-13T14:47:05-04:00', '2020-05-17T21:42:59-04:00', '2020-05-18T00:05:02-04:00', '2020-05-18T00:18:43-04:00', '2020-05-18T08:04:41-04:00', '2020-05-20T21:25:54-04:00', '2020-05-21T21:15:09-04:00', '2020-05-23T17:02:16-04:00', '2020-05-26T08:37:28-04:00', '2020-05-27T14:06:40-04:00', '2020-05-31T11:50:37-04:00', '2020-05-31T22:56:19-04:00', '2020-06-01T14:05:35-04:00', '2020-06-02T21:41:13-04:00', '2020-06-03T22:41:11-04:00', '2020-06-08T00:07:17-04:00', '2020-06-14T09:11:20-04:00', '2020-06-15T20:06:35-04:00', '2020-06-15T20:07:31-04:00', '2020-06-22T12:13:15-04:00', '2020-06-29T15:35:21-04:00', '2020-07-06T08:42:31-04:00'], 'title': 'Clean'}


In [103]:
print (time_stamp_array[2])

{'_id': ObjectId('5e9fbe235b2f8000240044a0'), 'cadence': 'Daily', 'cadenceCount': 1, 'creationDate': '2020-04-22T03:46:43.699Z', 'creatorID': ObjectId('5e9fb10585928b0024be726c'), 'timeStamps': ['2020-04-21T23:58:05-04:00', '2020-04-22T15:11:05-04:00', '2020-04-23T14:54:17-04:00', '2020-04-25T21:16:01-04:00', '2020-04-25T21:16:02-04:00', '2020-04-25T21:16:02-04:00', '2020-04-25T21:16:03-04:00', '2020-04-29T22:01:17-04:00', '2020-04-30T09:50:21-04:00', '2020-05-03T21:27:47-04:00', '2020-05-03T21:27:50-04:00', '2020-05-03T21:27:53-04:00', '2020-05-03T21:28:58-04:00', '2020-05-03T21:29:00-04:00', '2020-05-03T21:29:02-04:00', '2020-05-03T21:29:04-04:00', '2020-05-17T21:18:35-04:00', '2020-05-18T17:46:49-04:00', '2020-05-20T20:09:15-04:00', '2020-05-20T21:23:48-04:00', '2020-05-22T16:42:14-04:00', '2020-05-23T17:01:46-04:00', '2020-05-23T17:01:55-04:00', '2020-05-25T15:47:10-04:00', '2020-05-26T21:31:27-04:00', '2020-05-28T12:39:04-04:00', '2020-05-31T22:50:59-04:00', '2020-06-01T14:12:59-0

In [104]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'

In [ ]:
for i in users_array:
    try:
        print (i['email'])
    except:
        print (i)


In [ ]:
time_stamps_array_grouped_two = [[time_stamp_dict_create(time_stamp,D) for time_stamp in D['timeStamps']] for D in goals_array]
print (time_stamps_array_grouped_two[0])